In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import json
import torch
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from transformers import get_linear_schedule_with_warmup
from datasets import Dataset



In [ ]:
ENABLE_SUBMISSION = True  # Enable submission generation

## Config

In [ ]:
# model_name = '/kaggle/input/deberta-v3-small/transformers/default/1' V3 # current best
# model_name = "/kaggle/input/mdeberta-v3-base/mdeberta-v3-base" V4 # 2nd best
# model_name = "/kaggle/input/huggingfacedebertav3variants/deberta-v3-xsmall" # 3rd 
model_name = "/kaggle/input/huggingfacedebertav3variants/deberta-v3-large"
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model_path = "./results/full_training"

EPOCH = 15
FRAC = 1.0

## Load and sample dataset

In [ ]:
df = pd.read_csv("/kaggle/input/llm-classification-finetuning/train.csv")
df = df.sample(frac=FRAC, random_state=42).reset_index(drop=True)

def map_labels(row):
    if row["winner_model_a"] == 1:
        return 0
    elif row["winner_model_b"] == 1:
        return 1
    else:
        return 2

df["labels"] = df.apply(map_labels, axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
df_train, df_val = train_test_split(df, test_size=0.1, stratify=df["labels"], random_state=42)

## Tokenization

In [ ]:
def tokenize_batch(batch):
    text_a = [f"Prompt: {p} Response: {r}" for p, r in zip(batch["prompt"], batch["response_a"])]
    text_b = [f"Prompt: {p} Response: {r}" for p, r in zip(batch["prompt"], batch["response_b"])]
    return tokenizer(text_a, text_b, padding="max_length", truncation=True, max_length=256)

train_dataset = Dataset.from_pandas(df_train).map(tokenize_batch, batched=True, batch_size=500)
val_dataset = Dataset.from_pandas(df_val).map(tokenize_batch, batched=True, batch_size=500)

def clean(ds):
    return ds.remove_columns([col for col in ds.column_names if col not in ["input_ids", "attention_mask", "labels"]]).with_format("torch")

train_dataset = clean(train_dataset)
val_dataset = clean(val_dataset)

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted", zero_division=0)
    return {"accuracy": acc, "precision": precision, "recall": recall, "eval_f1": f1}


In [ ]:
# Load model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3).to(device)

In [ ]:
training_args = TrainingArguments(
    output_dir=model_path,
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=EPOCH,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=32,
    learning_rate=5e-6,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir=f"./logs/full_training",
    logging_steps=10,
    fp16=True,
    report_to="none",
    remove_unused_columns=False,
    load_best_model_at_end=True,
    metric_for_best_model="eval_f1",
    greater_is_better=True,
    label_smoothing_factor=0.1,
    lr_scheduler_type="cosine", 
    warmup_ratio=0.1,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

model.config.classifier_dropout_prob = 0.5


In [ ]:
trainer.train()
trainer.save_model(model_path)

In [ ]:
if ENABLE_SUBMISSION:
    test_df = pd.read_csv("/kaggle/input/llm-classification-finetuning/test.csv")
    test_ids = test_df["id"]
    test_dataset = Dataset.from_pandas(test_df).map(tokenize_batch, batched=True)
    test_dataset = test_dataset.remove_columns([col for col in test_dataset.column_names if col not in ["input_ids", "attention_mask"]]).with_format("torch")
    test_logits = trainer.predict(test_dataset).predictions
    test_probs = torch.softmax(torch.tensor(test_logits), dim=-1).cpu().numpy()
    submission = pd.DataFrame({
        "id": test_ids,
        "winner_model_a": test_probs[:, 0].round(6),
        "winner_model_b": test_probs[:, 1].round(6),
        "winner_tie": test_probs[:, 2].round(6)
    })
    submission.to_csv("/kaggle/working/submission.csv", index=False)
    print("✅ Submission file saved: submission.csv")